# Библиотеки, данные

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 11.4 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import evaluate
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from datasets import load_dataset
from datasets import Dataset

from transformers import (AutoModelForSequenceClassification,
                          AutoTokenizer,
                          TrainingArguments,
                          Trainer)

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/BIV хакатон/labeled_main_clean.csv')
train_df

,Unnamed: 0,text,labels
0,0,участие конференция address_tag,SERVICE
1,1,оказание услуга,SERVICE
2,2,порошок стиральный ariel color automat amount_tag,NON_FOOD_GOODS
3,3,возврат денежный средство договор заём no_nds_tag,LOAN
4,4,сброс загрязнять вещество,NOT_CLASSIFIED
...,...,...,...
25495,25495,сырье пвад yes_nds_tag,NON_FOOD_GOODS
25496,25496,гос услуга,TAX
25497,25497,предоставление кредит приобретение недвижимость,BANK_SERVICE
25498,25498,гос пошлина,TAX


In [ ]:
trg_encoder = LabelEncoder()
train_df['target'] = trg_encoder.fit_transform(train_df['labels'])
train_df

,Unnamed: 0,text,labels,target
0,0,участие конференция address_tag,SERVICE,7
1,1,оказание услуга,SERVICE,7
2,2,порошок стиральный ariel color automat amount_tag,NON_FOOD_GOODS,4
3,3,возврат денежный средство договор заём no_nds_tag,LOAN,3
4,4,сброс загрязнять вещество,NOT_CLASSIFIED,5
...,...,...,...,...
25495,25495,сырье пвад yes_nds_tag,NON_FOOD_GOODS,4
25496,25496,гос услуга,TAX,8
25497,25497,предоставление кредит приобретение недвижимость,BANK_SERVICE,0
25498,25498,гос пошлина,TAX,8


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    train_df['text'],
    train_df['target'],
    stratify=train_df['target'],
    random_state=42,
    test_size=0.3
)

In [ ]:
X_val

,text
21372,алименты
20837,оказание прочих услуга
15242,покупка непродовольственный товар связать пред...
12806,налог
1732,перец индана душистый горошек amount_tag
...,...
5229,гос пошлина
9678,мясной деликатес yes_nds_tag
10812,насос частичный yes_nds_tag
3744,бронирование отель address_tag уральский


In [ ]:
train_ds = Dataset.from_dict(
    {
        'text': list(X_train),
        'labels': list(y_train)
        }
)

val_ds = Dataset.from_dict(
    {
        'text': list(X_val),
        'labels': list(y_val)
     }
)

train_ds['text'][0]

'печение юбилейный традиционный витаминизированный amount_tag'

# Работа с моделью

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('moctarsmal/bank-transactions-statements-classification')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.97k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.45M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/574k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/250 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tkn_train_ds = train_ds.map(tokenize_function, batched=True)
tkn_val_ds = val_ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/17850 [00:00<?, ? examples/s]

Map:   0%|          | 0/7650 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    'moctarsmal/bank-transactions-statements-classification',
    num_labels=y_train.unique().shape[0],
    ignore_mismatched_sizes=True
)

config.json:   0%|          | 0.00/4.28k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/218M [00:00<?, ?B/s]

Some weights of FlaubertForSequenceClassification were not initialized from the model checkpoint at moctarsmal/bank-transactions-statements-classification and are newly initialized because the shapes did not match:
- sequence_summary.summary.bias: found shape torch.Size([59]) in the checkpoint and torch.Size([9]) in the model instantiated
- sequence_summary.summary.weight: found shape torch.Size([59, 512]) in the checkpoint and torch.Size([9, 512]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_args = TrainingArguments(output_dir="test_trainer",
                               eval_strategy="epoch")

In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tkn_train_ds,
    eval_dataset=tkn_val_ds,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.411900,0.312009,0.937516
2,0.240400,0.243435,0.955948
3,0.178700,0.171503,0.968758


TrainOutput(global_step=6696, training_loss=0.3923609026016728, metrics={'train_runtime': 1406.7361, 'train_samples_per_second': 38.067, 'train_steps_per_second': 4.76, 'total_flos': 3112436904192000.0, 'train_loss': 0.3923609026016728, 'epoch': 3.0})

In [ ]:
trainer.save_model('nice_model')